Import all necessary libraries and install everything you need for training:

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score, recall_score
import torch
from numba import cuda
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
#!pip install -q transformers

In [2]:
# Install the simpletransformers
#!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

In [3]:
# Install wandb
#!pip install -q wandb

In [4]:
import wandb

In [5]:
# Login to wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tajak (use `wandb login --relogin` to force relogin)


True

In [6]:
# Clean the GPU cache

cuda.select_device(0)
cuda.close()
cuda.select_device(0)
torch.cuda.empty_cache()


### Import the data

In [7]:
# GINCO full set (all labels except the labels with less than 10 instances)
train_df = pd.read_csv("data-splits/GINCO-downcast-train.csv", index_col = 0)
dev_df = pd.read_csv("data-splits/GINCO-downcast-dev.csv", index_col = 0)
test_df = pd.read_csv("data-splits/GINCO-downcast-test.csv", index_col = 0)

print("GINCO train shape: {}, Dev shape: {}, Test shape: {}.".format(train_df.shape, dev_df.shape, test_df.shape))

GINCO train shape: (601, 2), Dev shape: (201, 2), Test shape: (200, 2).


In [8]:
train_df.head()

,text,labels
324,"Ja tudi meni se zdi grdo, da moram razlagati z...",Forum
253,Bo kolesarjenje postalo del zimskih športov? <...,News/Reporting
674,Teptanje ustave namesto prevzema odgovornosti ...,Opinion/Argumentation
600,Cena odtisa za Canon i990 <p/> Zdaj smo končno...,Opinion/Argumentation
460,Veronikin tek je idealna priložnost za priprav...,Promotion


## Training and saving

We will use the multilingual XLM-RoBERTa model
https://huggingface.co/xlm-roberta-base

In [9]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()
LABELS

['Forum',
 'News/Reporting',
 'Opinion/Argumentation',
 'Promotion',
 'Information/Explanation',
 'List of Summaries/Excerpts',
 'Other',
 'Instruction',
 'Legal/Regulation']

In [11]:
# Initialize Wandb
wandb.init(project="GINCO-hyperparameter-search", name="GINCO-downcast-hyperparameter-search")

In [12]:
# Calculate how many steps will each epoch have
# Num steps in epoch = training samples / batch size
steps_per_epoch = int(601/8)
steps_per_epoch

75

I evaluated per every 10th epoch - per 750 steps. I first trained the model while evaluating it to find the optimal number of epochs. As previously already 20 epochs was enough, I will train only up to 30 epochs.

In [13]:
# Create a TransformerModel and evaluate during training
epoch = 30

roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": epoch,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            # Use these parameters if you want to evaluate during training
            "evaluate_during_training": True,
            "evaluate_during_training_steps": steps_per_epoch*10,
            "evaluate_during_training_verbose": True,
            "use_cached_eval_features": True,
            'reprocess_input_data': True,
            "labels_list": LABELS,
            # The following parameters (no_cache, no_save) are commented out if I want to save the model
            "no_cache": True,
            # Disable no_save: True if you want to save the model
            "no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'GINCO-hyperparameter-search',
            "silent": True,
            }
        )

# Train the model and evaluate during training
roberta_base_model.train_model(train_df, eval_df = dev_df)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

KeyboardInterrupt: 

Evaluation during training showed that the number of epochs before the eval_loss starts rising is somewhere between epochs 12 and 25. We then trained the model for epochs 12, 15, 20 and 25 to find the optimum number.

In [16]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("GINCO-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [14]:
# Open the main results file:

previous_results_file = open("results/GINCO-Experiments-Results.json")
previous_results = json.load(previous_results_file)
len(previous_results)

5

In [19]:
def testing(test_df, test_name, epoch):
    """
    This function takes the test dataset and applies the trained model on it to infer predictions.
    It also prints and saves a confusion matrix, calculates the F1 scores and saves the results in a list of results.

    Args:
    - test_df (pandas DataFrame)
    - test_name
    - epoch: num_train_epochs
    """
    # Get the true labels
    y_true = test_df.labels

    model = roberta_base_model
    
    # Calculate the model's predictions on test
    def make_prediction(input_string):
        return model.predict([input_string])[0][0]

    y_pred = test_df.text.apply(make_prediction)

    # Calculate the scores
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
    print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}")

    # Plot the confusion matrix:
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    plt.title(f"{test_name}")

    plt.tight_layout()
    fig1 = plt.gcf()
    plt.show()
    plt.draw()
    fig1.savefig(f"Confusion-matrix-{test_name}.png",dpi=100)

    # Save the results:
    rezdict = {
        "experiment": test_name,
        "num_train_epochs": epoch,
        "train_batch_size":8,
        "learning_rate": 1e-5,
        "microF1": micro,
        "macroF1": macro,
        "y_true": y_true.to_dict(),
        "y_pred": y_pred.to_dict(),
        }
    previous_results.append(rezdict)

    #Save intermediate results (just in case)
    backup = []
    backup.append(rezdict)
    with open(f"backup-results-{test_name}.json", "w") as backup_file:
        json.dump(backup,backup_file, indent= "")

In [21]:
# Train the model for various epochs to find the optimum number
epochs = [1, 12, 15, 20, 25]

for epoch in epochs:
    roberta_base_model = ClassificationModel(
                "xlmroberta", "xlm-roberta-base",
                num_labels=len(LABELS),
                use_cuda=True,
                args= {
                    "overwrite_output_dir": True,
                    "num_train_epochs": epoch,
                    "train_batch_size":8,
                    "learning_rate": 1e-5,
                    "labels_list": LABELS,
                    # The following parameters (no_cache, no_save) are commented out if I want to save the model
                    "no_cache": True,
                    # Disable no_save: True if you want to save the model
                    "no_save": True,
                    "max_seq_length": 512,
                    "save_steps": -1,
                    # Only the trained model will be saved - to prevent filling all of the space
                    "save_model_every_epoch":False,
                    "wandb_project": 'GINCO-hyperparameter-search',
                    "silent": True,
                    }
                )

    # Train the model
    roberta_base_model.train_model(train_df)
    
    # Test the model on dev_df
    testing(dev_df, f"Dev-epoch-search:{epoch}", epoch)

In [22]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

The experiments revealed that the optimum number of epochs is 20.

In [23]:
# Save the file with updated results.
with open("GINCO-Experiments-Results.json", "w") as results_file:
    json.dump(previous_results,results_file, indent= "")

Train the model and save it.

In [15]:
# Create a TransformerModel
roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=len(LABELS),
        use_cuda=True,
        args= {
            "overwrite_output_dir": True,
            "num_train_epochs": 20,
            "train_batch_size":8,
            "learning_rate": 1e-5,
            "labels_list": LABELS,
            # The following parameters are commented out because I want to save the model
            #"no_cache": True,
            # Disable no_save: True if you want to save the model
            #"no_save": True,
            "max_seq_length": 512,
            "save_steps": -1,
            # Only the trained model will be saved - to prevent filling all of the space
            "save_model_every_epoch":False,
            "wandb_project": 'GINCO-hyperparameter-search',
            "silent": True,
            }
        )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [16]:
# Train the model
roberta_base_model.train_model(train_df)

/home/tajak/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


(1460, 1.138362426320984)

In [17]:
# Save the trained model to Wandb
run = wandb.init(project="GINCO-hyperparameter-search", entity="tajak", name="saving-trained-model")
trained_model_artifact = wandb.Artifact("GINCO-full-set-classifier", type="model", description="a model trained on the (Slovene) GINCO dataset with the full set of labels (only the labels with less than 10 instances were excluded. There are 17 labels: 'Promotion of a Product', 'Forum', 'Instruction', 'News/Reporting', 'Opinionated News', 'Other', 'List of Summaries/Excerpts', 'Invitation', 'Opinion/Argumentation', 'Legal/Regulation', 'Information/Explanation', 'Review', 'Promotion of Services', 'Promotion', 'Announcement', 'Call', 'Correspondence'")
trained_model_artifact.add_dir("outputs")
run.log_artifact(trained_model_artifact)

Training loss,██▇▆█▅▄▄▃▆▃▃▁▃▃▂▃▂▂▁▂▂▃▁▂▁▁▁▁
global_step,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
lr,▅██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
Training loss,0.31044
global_step,1450
lr,0.0


wandb: Adding directory to artifact (./outputs)... Done. 2.8s
